In [2]:
import numpy as np

import sklearn.svm as ssv

from sklearn.externals import joblib

from skimage.feature import hog

import random

import glob

import os

import cv2

In [3]:
def trainSvm(datas, labels):

    clf = ssv.SVC(kernel='linear', C = 0.6 , gamma = 20, decision_function_shape='ovo')

    print ("Training a SVM")

    clf.fit(datas, labels)

    score = clf.score(datas, labels)
    
    print (score)
    
    return clf,score

In [23]:
def sliding_window(image, stepSize, windowSize):

    for y in range(0, 224, stepSize):

        for x in range(0, 224, stepSize):
            
            y = y + windowSize[1]
            
            x = x + windowSize[0]

            yield (x, y, image[x,y])

In [24]:
def rgb2gray(im):

    gray = im[:, :, 0]*0.2989+im[:, :, 1]*0.5870+im[:, :, 2]*0.1140

    return gray

In [25]:
if __name__ == "__main__":

    normalize = True
    
    visualize = False

    block_norm = 'L2-Hys'

    cells_per_block = [2,2]

    pixels_per_cell = [20,20]

    orientations = 9

    new_model_path = './models/svm'
    
    record = []
    
    negative_img_path = 'C:/ZZF/SVM/VOCtrainval_11-May-2012/VOCdevkit/VOC2012/negative'

    num = 0

    for childDir in os.listdir(negative_img_path):

        num += 1

        num_win = 0

        #print "num: %d hard negative mining: %s" %(num,childDir)

        f = os.path.join(negative_img_path, childDir)

        data = cv2.imread(f)

        scales = [(100, 100), (200,200), (300,300), (400,400), (500, 500), (600,600), (800, 800)]

        for (winW,winH) in scales:

            for (x, y, window) in sliding_window(data, stepSize=100, windowSize=(winW,winH)):

                result = 0

                if window.shape[0] != winH or window.shape[1] != winW:

                    continue

                if window.shape[0] != 200 or window.shape[1] != 200:

                    window = cv2.resize(window,(200,200),interpolation=cv2.INTER_CUBIC)

                gray = rgb2gray(window)/255.0

                window_fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm, visualize, normalize)

                win_fd = window_fd.reshape(1, -1)

                result = int(clf.predict(win_fd))

                if result >= 0.5:
                    
                    record.append((window_fd, result))

                   
    New_clf = clf(record)

    
    joblib.dump(new_clf, new_model_path)

    print ("Classifier saved to {}".format(new_model_path))
    







  

    


    

    

TypeError: 'NoneType' object is not subscriptable